In [1]:
from sklearn.datasets import make_regression
import pandas as pd 
import numpy as np 

In [2]:
X, y = make_regression(n_samples= 300, n_features=3, noise=15, random_state=42)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3'])
y = pd.Series(y)

#### Скорость обучения

нужно добавить изменение шага обучения 

In [97]:
'''
class MyLineReg():
    def __init__(self, n_iter=100, learning_rate=0.1, reg=None, l1_coef=0, l2_coef=0, metric=None, weights=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.weights = weights
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.best_score = None

        # Проверям на входные параметры подсчета метрики
        if self.metric is not None and self.metric not in ['mae', 'mse', 'rmse', 'mape', 'r2']:
            raise ValueError("metric must be 'mae', 'mse', 'rmse', 'mape' or 'r2'")
        # Проверям на входные параметры регулиризации 
        if self.reg is not None and self.reg not in ['l1', 'l2', 'elasticnet']:
            raise ValueError("reg must be 'l1', 'l2', or 'elasticnet'")
        # Проверка коэффициентов (должны быть от 0 до 1)
        if self.l1_coef < 0 or self.l1_coef > 1:
            raise ValueError("l1_coef must be in [0, 1]")
        if self.l2_coef < 0 or self.l2_coef > 1:
            raise ValueError("l2_coef must be in [0, 1]")

    def __str__(self):
        return f'MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'

    # Напишем метод с приватным модификатором доступа для подсчета разнных метрик 
    def __calculate_metric(self, pred: pd.DataFrame, y_true: pd.Series, metric:'str'):
        """
        Вычисляет значение метрики.

        Параметры:
        pred (np.array): Предсказанные значения.
        y_true (np.array): Фактические значения.
        metric (str): Название метрики.

        Возвращает:
        float: Значение метрики.
        """
        if metric == 'mae':
            return np.mean(np.abs(pred - y_true))
        elif metric == 'mse':
            return np.mean((pred - y_true)**2)
        elif metric == 'rmse':
            return np.sqrt(np.mean((pred - y_true)**2))
        elif metric == 'mape':
            return 100 * np.mean(np.abs((pred - y_true) / y_true))
        elif metric == 'r2':
            ss_residual = np.sum((pred - y_true)**2)
            ss_total = np.sum((y_true - np.mean(y_true))**2)
            return 1 - (ss_residual / ss_total)
        else:
            raise ValueError(f"Unknown metric: {metric}")
    def fit(self, X: pd.DataFrame, y:pd.Series, verbose=False):
        """
        Обучение модели линейной регрессии.

        Параметры:
        X (pd.DataFrame): Матрица признаков.
        y (pd.Series): Вектор целевых значений.
        verbose (bool): Флаг для вывода информации о процессе обучения.
        """
        X_copy = X.copy()               # создадим копию нашей матрицы фичей, что бы не изменить изначальный
        X_copy.insert(0, 'base', 1)     # допишем слева столбик из 1 для свободного члела
        w = np.ones(X_copy.shape[1])    # составим вектор весов, заполненный из 1

        # Напишем подсчет ошибки до начала обучения
        if verbose:                   
            initial_MSE = np.mean((np.dot(X_copy, w) - y)**2)          # ошибка MSE + L1 
            if self.reg == 'l1':
                initial_MSE += self.l1_coef * np.sum(np.abs(w))    # ошибка MSE + L2
            if self.reg == 'l2':
                initial_MSE += self.l2_coef * np.sum(w**2)         # ошибка MSE + elasticnet
            if self.reg == 'elasticnet':
                initial_MSE += self.l1_coef * np.sum(np.abs(w)) + self.l2_coef * np.sum(w**2)
                
            if self.metric is None:                          # Если не задана метрика, то в логи выводим только MSE
                print(f'start | loss: {initial_MSE:0.2f}')   
            else:                                            # Если задана, то выводим ошибку на MSE и на выбранной нами метрике
                print(f'start | loss: {initial_MSE:0.2f} | {self.metric}: {self.__calculate_metric(pred=X_copy.dot(w), y_true=y, metric=self.metric):0.2f}')
            
        for i in range(self.n_iter): # Напишем цикл обучения
            pred = np.dot(X_copy, w) # cчитаем предасказания модели 
            error = pred - y         # вычисляем ошибку (предсказания - реальные значения)
            grad = (2/len(X_copy)) * np.dot(error, X_copy) # вычисляем градиенты по весам  (Градиент MSE)

            # добавим градиент регуляризации к весам модели, если она установлена
            if self.reg == 'l1':                                         # градиент MSE с L1 регуляризацией 
                grad += self.l1_coef * np.sign(w)
            elif self.reg == 'l2':                                       # градиент MSE с L2 регуляризацией 
                grad += 2 * self.l2_coef * w 
            elif self.reg == 'elasticnet':                               # градиент MSE c elasticnet регуляризацией
                grad += self.l1_coef * np.sign(w) + 2 * self.l2_coef * w
                
            if self.metric:         # Если задана метрика, то считаем его и обновляем self.best_score
                metric_loss = self.__calculate_metric(pred = X_copy.dot(w), y_true=y, metric=self.metric) # Считаем выбранную метрику
                self.best_score = metric_loss

            if verbose:     # выводим логи 
                loss = np.mean(error**2)
                if self.reg == 'l1':                             # Если L1 то к ошибке MSE + L1
                    loss += self.l1_coef * np.sum(np.abs(w)) 
                if self.reg == 'l2':                            
                    loss += self.l2_coef * np.sum((w)**2)   # Если L2 то к ошибке MSE + L2
                if self.reg == 'elasticnet':                     # Есле elasticnet то к ошибке MSE + elasticnet
                    loss += self.l1_coef * np.sum(np.abs(w)) + self.l2_coef * np.sum((w)**2)
                if self.metric is None and i % 10 == 0:        # Eсли метрика не указана, то в логи выыодим итерацию и ошибку на MSE
                    print(f'Iteration {i}| loss: {loss}')
                elif self.metric and (i % 10 == 0 or self.n_iter - 1 == i): # Если указана метрика, то выводим ошибку MSE и на метрике
                    print(f'Iteration {i}| loss: {loss:0.2f} | {self.metric}: {metric_loss:0.2f}')
            if callable(self.learning_rate):        
                w -= self.learning_rate(i+1) * grad  # обновляем веса, обновляем тут, чтобы ошибка считалась правильно     
            else:
                w -= self.learning_rate * grad
        # сохроняем веса
        self.weights = w 
    def predict(self, X: pd.DataFrame)->np.array:
        """
        Предсказание целевых значений.

        Параметры:
        X (pd.DataFrame): Матрица признаков.

        Возвращает:
        np.array: Предсказанные значения.
        """
        X_copy = X.copy()            # делаем копию матрицы фичей, чтобы не менять изначальный датафрейм
        X_copy.insert(0, 'base', 1)  # допишем слева столбик из 1 для свободного члена
        predict = X_copy.dot(self.weights) # делаем предсказания X_copy @ self.weights
        return predict 

    def get_best_score(self):  # Метод для вывода лучшего качетва на выбранной метрике
        if self.metric is None:
            raise ValueError("Metric was not set during model initialization.")
        return self.best_score

    def get_coef(self):
        return self.weights[1:] # метод для вывода весов начиная с 1го заначения            
''';

Измененная улучшенная версия модели

In [57]:
class MyLineReg():
    def __init__(self, n_iter=100, learning_rate=0.1, reg=None, l1_coef=0, l2_coef=0, metric=None, weights=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.weights = weights
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.best_score = None

        self.__validate_params() # проверка входных параметров

    def __str__(self):
        return f'MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'
        
    def __validate_params(self):  # Проверяем на корректность входные параметры 
        # Проверям на входные параметры подсчета метрики
        if self.metric is not None and self.metric not in ['mae', 'mse', 'rmse', 'mape', 'r2']:
            raise ValueError("metric must be 'mae', 'mse', 'rmse', 'mape' or 'r2'")
        # Проверям на входные параметры регулиризации 
        if self.reg is not None and self.reg not in ['l1', 'l2', 'elasticnet']:
            raise ValueError("reg must be 'l1', 'l2', or 'elasticnet'")
        # Проверка коэффициентов (должны быть от 0 до 1)
        if self.l1_coef < 0 or self.l1_coef > 1:
            raise ValueError("l1_coef must be in [0, 1]")
        if self.l2_coef < 0 or self.l2_coef > 1:
            raise ValueError("l2_coef must be in [0, 1]")

    # Напишем метод с приватным модификатором доступа для подсчета разнных метрик 
    def __calculate_metric(self, pred: np.array, y_true: np.array, metric: str) -> float:
        """
        Вычисляет значение метрики.

        Параметры:
        pred (np.array): Предсказанные значения.
        y_true (np.array): Фактические значения.
        metric (str): Название метрики.

        Возвращает:
        float: Значение метрики.
        """
        if metric == 'mae':
            return np.mean(np.abs(pred - y_true))
        elif metric == 'mse':
            return np.mean((pred - y_true)**2)
        elif metric == 'rmse':
            return np.sqrt(np.mean((pred - y_true)**2))
        elif metric == 'mape':
            return 100 * np.mean(np.abs((pred - y_true) / y_true))
        elif metric == 'r2':
            ss_residual = np.sum((pred - y_true)**2)
            ss_total = np.sum((y_true - np.mean(y_true))**2)
            return 1 - (ss_residual / ss_total)
        else:
            raise ValueError(f"Unknown metric: {metric}")
            
    def __calculate_loss(self, X: np.array, y: np.array, w: np.array) -> float:       
        '''
        Вычисляем общий loss (MSE + регуляризация).
        X: np.array (Матрица признаков)
        y: np.array (Вектор значений)
        w: np.array (Вектор весов)
        '''
        error = np.dot(X,w) - y
        MSE = np.mean(error**2) 
        
        # Добавим регуляризацию, если она установлена
        if self.reg == 'l1':
            return MSE + self.l1_coef * np.sum(np.abs(w))
        elif self.reg == 'l2':
            return MSE + self.l2_coef * np.sum(w**2)
        elif self.reg == 'elasticnet':
            return MSE + self.l1_coef * np.sum(np.abs(w)) + self.l2_coef * np.sum(w**2)
        else:
            return MSE

    def __log_training_step(self, iteration: int, X: np.array, y: np.array, w: np.array):
        """Логирует процесс обучения."""
        loss = self.__calculate_loss(X, y, w)
        if self.metric:
            metric_value = self.__calculate_metric(X.dot(w), y, self.metric)
            print(f'Iteration {iteration} | loss: {loss:.2f} | {self.metric}: {metric_value:.2f}')
        else:
            print(f'Iteration {iteration} | loss: {loss:.2f}')

    def __apply_regularization(self, grad: np.array, w: np.array) -> np.array:
        """Добавляет градиент регуляризации."""
        if self.reg == 'l1':
            grad += self.l1_coef * np.sign(w)
        elif self.reg == 'l2':
            grad += 2 * self.l2_coef * w
        elif self.reg == 'elasticnet':
            grad += self.l1_coef * np.sign(w) + 2 * self.l2_coef * w
        return grad

    def __get_learning_rate(self, iteration: int) -> float:
        """Вычисляет learning_rate (статический или динамический)."""
        if callable(self.learning_rate):
            return self.learning_rate(iteration)
        else:
            return self.learning_rate
        
    def fit(self, X: pd.DataFrame, y:pd.Series, verbose=False):
        """
        Обучение модели линейной регрессии.

        Параметры:
        X (pd.DataFrame): Матрица признаков.
        y (pd.Series): Вектор целевых значений.
        verbose (bool): Флаг для вывода информации о процессе обучения.
        """
        X_copy = X.copy()               # создадим копию нашей матрицы фичей, что бы не изменить изначальный
        X_copy.insert(0, 'base', 1)     # допишем слева столбик из 1 для свободного члела
        w = np.ones(X_copy.shape[1])    # составим вектор весов, заполненный из 1

        if verbose:
            loss = self.__calculate_loss(X_copy, y, w) # вычисляем loss
            if self.metric is None:
                print(f'start | loss: {loss:.2f}')
            else:
                metric = self.__calculate_metric(pred=X_copy.dot(w), y_true=y, metric=self.metric)
                print(f'start | loss: {loss:0.2f} | {self.metric}: {metric:0.2f}')

        # Напишем цикл обучения
        for i in range(self.n_iter):
            pred = X_copy.dot(w)                         # считаем предсказания модели
            error = pred - y                             # вычислим ошибку (предсказания - реальные значения)
            grad = (2/len(y)) * np.dot(error, X_copy)    # вычислим градиент по весам модели
            grad = self.__apply_regularization(grad, w)  # добавим регуляризацию, если она задана

            if self.metric:                              # Если задана метрика, то считаем её и обновляем self.best_score
                self.best_score = self.__calculate_metric(pred=pred, y_true=y, metric=self.metric)
            
            if verbose and (i % 10 == 0 or i == self.n_iter - 1): # Eсли задано логирование, то выводим логи
                self.__log_training_step(i, X_copy, y, w)

            lr = self.__get_learning_rate(i+1)                    # проверяем на динамичность lr
            w -= lr* grad                                         # делаем шаг обучения
            
        self.weights = w    # сохроняем веса

    def predict(self, X: pd.DataFrame)->np.array:
        """
        Предсказание целевых значений.

        Параметры:
        X (pd.DataFrame): Матрица признаков.

        Возвращает:
        np.array: Предсказанные значения.
        """
        X_copy = X.copy()            # делаем копию матрицы фичей, чтобы не менять изначальный датафрейм
        X_copy.insert(0, 'base', 1)  # допишем слева столбик из 1 для свободного члена
        predict = X_copy.dot(self.weights) # делаем предсказания X_copy @ self.weights
        return predict 

    def get_best_score(self):  # Метод для вывода лучшего качетва на выбранной метрике
        if self.metric is None:
            raise ValueError("Metric was not set during model initialization.")
        return self.best_score

    def get_coef(self):
        return self.weights[1:] # метод для вывода весов начиная с 1го заначения            


In [61]:
lr = lambda iter: 0.5 * (0.85 ** iter)

In [63]:
lr(0.1)

0.49193972827026317

In [65]:
model_lr = MyLineReg(n_iter=30,learning_rate=lr, metric='mse')

In [67]:
model_lr.fit(X, y, verbose=True)

start | loss: 13352.60 | mse: 13352.60
Iteration 0 | loss: 13352.60 | mse: 13352.60
Iteration 10 | loss: 203.40 | mse: 203.40
Iteration 20 | loss: 203.37 | mse: 203.37
Iteration 29 | loss: 203.37 | mse: 203.37


In [69]:
model = MyLineReg(n_iter=300,metric='mse', reg='elasticnet', l1_coef=0.9, l2_coef=0)

In [71]:
model.fit(X, y, verbose=True)

start | loss: 13356.20 | mse: 13352.60
Iteration 0 | loss: 13356.20 | mse: 13352.60
Iteration 10 | loss: 530.77 | mse: 388.16
Iteration 20 | loss: 367.58 | mse: 208.65
Iteration 30 | loss: 365.41 | mse: 204.52
Iteration 40 | loss: 365.38 | mse: 204.25
Iteration 50 | loss: 365.38 | mse: 204.22
Iteration 60 | loss: 365.38 | mse: 204.21
Iteration 70 | loss: 365.38 | mse: 204.21
Iteration 80 | loss: 365.38 | mse: 204.21
Iteration 90 | loss: 365.38 | mse: 204.21
Iteration 100 | loss: 365.38 | mse: 204.21
Iteration 110 | loss: 365.38 | mse: 204.21
Iteration 120 | loss: 365.38 | mse: 204.21
Iteration 130 | loss: 365.38 | mse: 204.21
Iteration 140 | loss: 365.38 | mse: 204.21
Iteration 150 | loss: 365.38 | mse: 204.21
Iteration 160 | loss: 365.38 | mse: 204.21
Iteration 170 | loss: 365.38 | mse: 204.21
Iteration 180 | loss: 365.38 | mse: 204.21
Iteration 190 | loss: 365.38 | mse: 204.21
Iteration 200 | loss: 365.38 | mse: 204.21
Iteration 210 | loss: 365.38 | mse: 204.21
Iteration 220 | loss: 

In [93]:
model.get_coef()

array([82.21476195, 81.08657754, 15.70601866])